Import libraries and directories

In [12]:
import cv2
import numpy as np
import os

input_folder = '../Input_Images'
output_folder = '../Output_Images'
edge_folder = '../Edge_Maps'

Detect edges

In [13]:
def detect_edges(image):
    """
    Detects edges in the input grayscale image using Canny edge detection.
    Returns the edge-detected image.
    """
    blurred = cv2.GaussianBlur(image, (9, 9), 2)
    edges = cv2.Canny(blurred, 50, 150)
    return edges

Draw circles for pupil-iris and iris-sclera

In [14]:
def detect_circles(image, offset=60):
    output_image = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)
    # Detect the pupil
    _, thresholded_pupil = cv2.threshold(image, 50, 255, cv2.THRESH_BINARY_INV)
    contours, _ = cv2.findContours(thresholded_pupil, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if contours:
        largest_contour = max(contours, key=cv2.contourArea)
        (x, y), pupil_radius = cv2.minEnclosingCircle(largest_contour)
        center_pupil = (int(x), int(y))
        pupil_radius = int(pupil_radius)
        cv2.circle(output_image, center_pupil, pupil_radius, (0, 255, 0), 2)  # Green circle for pupil

        # iris-sclera boundary 
        iris_radius = pupil_radius + offset
        cv2.circle(output_image, center_pupil, iris_radius, (255, 0, 0), 2)  # Blue circle for iris-sclera

    return output_image

Save and process images

In [15]:
def process_and_save(image_path, edge_folder, output_folder):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    image_name = os.path.splitext(os.path.basename(image_path))[0]
    
    # Detect edges and save the edge map
    edges = detect_edges(image)
    edge_map_path = os.path.join(edge_folder, f"{image_name}_edge.tif")
    cv2.imwrite(edge_map_path, edges)
    
    # Detect circles for pupil and iris-sclera boundary
    output_image = detect_circles(image, offset=60)
    
    # Save the output image with detected circles
    output_image_path = os.path.join(output_folder, f"{image_name}_output.tif")
    cv2.imwrite(output_image_path, output_image)


# Process each image in the Input_Images folder
for filename in os.listdir(input_folder):
    if filename.endswith(".tif"):
        image_path = os.path.join(input_folder, filename)
        process_and_save(image_path, edge_folder, output_folder)

print("All images saved.")

All images saved.
